In [1]:
import pandas as pd
from glob import glob

In [ ]:
def merge_ravens(outputs_dir:str="/Users/shai/personal/deepvoice/amber/results_dec4/outputs/500_test2_amber"):
    d = {}
    for file in glob(outputs_dir):
        typ = file.split('-')[-1].split(".")[0]
        org_file = file.split('-')[0]
        
        df = pd.read_csv(file, sep='\t')
        if "bark" in typ:
            df['High Freq (Hz)'] = 3000
        else:
            df['Low Freq (Hz)'] = 1000
        
        if org_file in d.keys():
            d[org_file] = pd.concat([d[org_file], df], axis=1)
        else:
            d[org_file] = df

In [2]:
def add_data_to_df(df, name):
    final = pd.DataFrame()
    th = {
        'whistle': 0.5,
        'barks': 0.1,
        'buzz': 0.8,
        'echo': 0.9
    }[name]
    names, counts = [], []
    per_file = df.groupby('filename')
    for name_g, g in per_file:
        calls = g[name] > th
        names.append(name_g)
        counts.append(calls.value_counts().get(True, 0))
    final['filename'] = names
    final[f'count_{name}'] = counts
    return final


In [3]:
dfs = []
for file in glob('/Users/shai//personal/deepvoice/amber/results_dec4/outputs/barks_test1/*.csv'):
    df = pd.read_csv(file)
    df['filename'] = df['filename'].str.split('/').str[-1].str.replace('.wav','')
    
    name = file.split('-500')[0].split('_')[-1]
    if '4af2w6lt' in file:
        name = "barks"
    dfs.append(add_data_to_df(df, name))

/Users/shai/personal/deepvoice/soundbay/.env_dv/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.


In [4]:
final = dfs[0]
for d in dfs:
    final = pd.merge(final, d, on="filename", how="outer")
final

,filename,count_barks_x,count_barks_y,count_barks
0,6423.211103000010,0,0,0
1,6423.211103002010,0,0,0
2,6423.211103004010,1,1,0
3,6423.211103010010,1,1,0
4,6423.211103012010,1,1,0
...,...,...,...,...
495,6423.211109210010,0,0,0
496,6423.211109212010,0,0,0
497,6423.211109214010,1,1,0
498,6423.211109220010,1,1,0


In [5]:
# final = final[['filename','count_whistle','count_whistle_x','count_barks','count_echo','count_buzz']] 

In [6]:
final.to_excel('/Users/shai/personal/deepvoice/amber/results_dec4/amber_results_additional_barks_test1.xlsx', index=False)